## СКОЛЬЗЯЩЕЕ СРЕДНЕЕ

В предыдущем модуле мы уже знакомились с экспоненциальным сглаживанием и получением с его помощью прогноза. Помимо простого экспоненциального сглаживания, есть также простое сглаживание, или скользящее среднее.

Для получения элементарного случая скользящего среднего проще всего взять среднее арифметическое двух последних наблюдений. Мы получим новый временной ряд, каждый член которого — среднее арифметическое двух соседних значений исходного ряда:

$MA_t= \frac{X_{t-1}+X_t}{2}$

Чуть более продвинутый способ — усреднить сразу несколько наблюдений. Это так называемое простое скользящее среднее (`Simple Moving Average`, `SMA`):

$SMA_t= \frac{X_{t-q+1}+...+X_t}{q}$

Таким образом, в скользящем среднем мы суммируем несколько последовательных точек временного ряда и делим эту сумму на количество самих точек, то есть считаем математическое усреднение за определённый период.

Для сглаживания мы будем использовать встроенный метод [pandas.Series.rolling()](https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html) — он принимает на вход параметр window и ожидает после себя агрегирующую функцию для сглаживания (обычно используется среднее). Из преимуществ этого метода можно отметить простоту реализации и интерпретации, из недостатков — чувствительность.

Посмотрим на некоторый временной ряд и результаты применения сглаживания к нему:

<img src=m13_img1.png width=400>

Скользящее среднее с узким окном (размер окна — два дня) неэффективно борется с выбросами.

<img src=m13_img2.png width=400>

Скользящее среднее с широким окном (размер окна — 30 дней) может привести к потере информации, сгладив полезную информацию.

<img src=m13_img3.png width=400>

На данном графике с окном размера 15 есть прослеживающийся период и изменение амплитуды с течением времени.


*** 

## ARMA И ARIMA

Что получится, если объединить некоторые из изученных нами методов? Уже есть предположения, что такое `ARMA`?

Если вы внимательно следили за ходом событий, то смогли догадаться, что `ARMA` — это авторегрессионное скользящее среднее, или модель авторегрессии-скользящего среднего. В ней p авторегрессионных слагаемых и q слагаемых скользящего среднего шумовой компоненты:

$X_t = \alpha + \phi_1 X_{t-1} + \phi_2 X_{t-2} + \dots + \epsilon_t + \theta_{t-1}\epsilon_{t-1} + \dots + \theta_{q}\epsilon_{t-q}$

$p$ - параметр авторегрессионной модели

$q$ - параметр скользящего среднего

>Параметр p мы определяли по графику частичной автокорреляции. Параметр q для скользящего среднего определяют так же, но по коррелограмме (графику автокорреляции).

`ARIMA` расшифровывается как `Autoregressive Integrated Moving Average` и включает в себя ещё один параметр ($d$), который означает, что дифференцирование временного ряда порядка $d$ приводит ряд к стационарности и будет подчиняться модели `ARMA`.

>$d$ — это тот самый порядок дифференцирования из предыдущего модуля, который приводил нестационарный ряд к стационарности. Это значит, что даже если наш ряд нестационарный, мы можем сделать его стационарным путём взятия разностей. Запомнив получившееся количество дифференцирований, можно смело применять к нему `ARIMA`.

И `ARMA`, и `ARIMA` реализованы на `Python` в классе [ARIMA](https://www.statsmodels.org/devel/generated/statsmodels.tsa.arima.model.ARIMA.html) из `statsmodels`. Данному классу необходимо передать в качестве параметров временной ряд и порядок order `(ARIMA(dta, order=(2, 0, 0)))`. Для параметра `order` нужно указать `p`, `d` и `q` (именно в таком порядке), причём для получения `ARMA` необходимо указать `d=0`.

## Резюмируем:

* Если ряд **стационарный**, используем `ARMA`.

* Если ряд **нестационарный** (имеет тренд), с помощью дифференцирования определяем порядок d и используем `ARIMA`.
Теперь мы можем проверять временные ряды из задач на стационарность и вне зависимости от результата применять к ним одну из статистических моделей.

***

## SARIMA

Модель `ARIMA` отлично учитывает и тренд (благодаря скользящему среднему), и зависимость от предыдущих значений (благодаря авторегрессии), но в ней не хватает учёта сезонности. В таком случае можно добавить к ARIMA учёт сезонности, и тогда мы получим следующую модель — сезонную `ARIMA`, или `SARIMA` (S`easonal ARIMA`).

Эта модель очень похожа на ARIMA, за исключением того, что в ней есть дополнительный набор компонентов авторегрессии и скользящего среднего.

>`SARIMA` позволяет различать данные по сезонной частоте, а также по их несезонным отличиям. Нахождение лучших для модели параметров можно упростить с помощью средств автоматического поиска, таких как [auto_arima](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html) из [pmdarima](http://alkaline-ml.com/pmdarima/).

## SARIMAX И ARIMAX

Вы уже могли заметить, что изученные нами статистические модели совершают предсказания, основываясь только на данных самого временного ряда. Но что если месяц или день недели тоже имеет значение? Или, например, на курс рубля, помимо даты и предыдущих значений, также влияют курсы других валют?

Последней статистической моделью, с которой мы познакомимся, будет SARIMAX. Её отличие от предыдущей версии заключается в том, что, помимо данных временного ряда, она учитывает экзогенные переменные. Таким образом мы сможем учитывать не только зависимости внутри данных, но и внешние факторы.

Для запуска моделей SARIMA и SARIMAX на Python нужно воспользоваться классом [SARIMAX](https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html). Если вы хотите использовать SARIMA, необходимо задать два обязательных параметра — order и seasonal_order.

`order` — это порядок для модели (ARIMA(p, d, q)). В `seasonal_order` необходимо передать ещё четыре параметра:

* `P` — сезонный авторегрессионный порядок;
* `D` — порядок дифференцирования сезонного ряда;
* `Q` — порядок сезонной скользящей средней;
* `m` — размер сезонного периода.

Если размер сезонного периода m можно определить по сезонной компоненте (мы уже раскладывали ряд на компоненты ранее — [seasonal_decompose()](https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html)), то остальные параметры удобнее определять автоперебором (мы применим этот способ на практике в следующем юните).

Для учёта экзогенных переменных необходимо передать в класс SARIMAX параметр exog=x. В x должны находиться другие временные ряды, например курс доллара (x), который может влиять на курс рубля (y), или пометка, является ли каждая из дат праздничным днём.

Хороший пример реализации SARIMAХ приведён в [официальной документации](https://www.statsmodels.org/stable/examples/notebooks/generated/statespace_sarimax_faq.html).

>Обратите внимание, что также существует модель `ARIMAX`. Уже есть предположения, когда использовать эту модель? В отличие от SARIMAX, ARIMAX не учитывает сезонную составляющую, но имеет все преимущества ARIMA и учитывает экзогенные переменные.

## КАК СРАВНИВАТЬ ЭТИ МОДЕЛИ?

Одним из распространённых способов является сравнение качества моделей по **критерию Акаике** (`AIC`). Этот информационный критерий вознаграждает модель за качество приближения обученного временного ряда к фактическому, а также «штрафует» её за использование излишнего количества параметров. Принято считать, что модель с наименьшим значением критерия `AIC` является наилучшей.

Для оценки модели критерием AIC необязательно пользоваться дополнительными методами. Этот критерий, как и другая информация, отображается после обучения модели при вызове встроенного метода `fit_model.summary()`.

## КАК ВЫБРАТЬ ПОДХОДЯЩУЮ МОДЕЛЬ?

<img src=m13_img4.png>

## ДОПОЛНИТЕЛЬНО:

* Оценка качества модели информационными критериями продемонстрирована в [этой статье](https://www.projectpro.io/article/how-to-build-arima-model-in-python/544).

* Ещё один пример с расчётом критерия для различных комбинаций параметров представлен [здесь](https://www.8host.com/blog/prognozirovanie-vremennyx-ryadov-s-pomoshhyu-arima-v-python-3/).

## Интерполяция и сэмплирование. Практика

1. Проверьте данные на наличие пропущенных дат. Помимо визуального способа, это можно сделать с помощью метода [DataFrame.asfreq()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.asfreq.html), передав в него параметр частоты, например `‘d’` — день, `‘m’` — месяц. Все алиасы для параметров частоты доступны [по ссылке](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases).
Подсказка
```Python

    df.asfreq(freq='AS')
```
2. Проверьте данные на наличие пропущенных значений (`Series.isna().sum()`).

3. Обратите внимание, что иногда пропуски в данных могут быть заполнены нулями. Сделайте проверку на наличие нулевых значений и в случае нахождения замените их на NaN. Воспользуйтесь кодом ниже:
``` Python
    import numpy as np
    df['GDP (current US$)'] = df['GDP (current US$)'].apply(lambda x: np.nan if x==0 else x)
```
4. Для заполнения пропусков выполните интерполяцию с использованием метода .interpolate().
Подсказка
``` Python
    df['GDP (current US$)'].interpolate(method='linear')
```
5. Проверьте полученный ряд на стационарность, определите параметры модели (`ARIMA/ARMA`) и запустите модель.

6. Изменился ли `AIC` критерий построенной модели по сравнению с моделью на неинтерполированных данных? Сделайте вывод.

## Файл `project-Part2.ipynb`

## Модели прогнозирования гетероскедастичности

>**Дисперсия** — это статистический показатель, показывающий меру разброса данных вокруг среднего значения.

Если мы проведём на графике усреднённую прямую (красный пунктир), будет заметно, что ближе к 1997 году точки имеют больший разброс, чем в начале, то есть дисперсия данных изменчива во времени.

Подобная неоднородность наблюдений, выражающаяся в неодинаковой дисперсии, называется гетероскедастичностью.

Такой характер дисперсии можно часто встретить во временных рядах из финансовой сферы, поскольку они более подвержены гетероскедастичности.

Отсутствие гетероскедастичности называется гомоскедастичностью.

Сравните графики временных рядов ниже:

| Гомоскедастичный временной ряд | Гетероскедастичный временной ряд |
| - | - |
| <img src=m13_img5.png width=400> | <img src=m13_img6.png width=400> |
| <img src=m13_img7.png width=400> | <img src=m13_img8.png width=400> |
| <img src=m13_img9.png width=400> | <img src=m13_img10.png width=400> |

Изученные нами модели помогают учесть тренд и сезонность. **Но что насчёт дисперсии?** Обычно, если временной ряд обладает гетероскедастичностью, такие модели не способны уловить изменение дисперсии во времени, и их предсказания выглядят примерно так:

<img src=m13_img11.png width=800>

Видно, что в предсказании мы не учли разброс — прогноз хочется «вытянуть» вверх, таким образом увеличивая дисперсию. Такая проблема часто возникает в волатильных временных рядах. Понятие волатильности также относится к финансовым временным рядам и является одним из самых важных финансовых показателей и понятий в управлении финансовыми рисками.

>**Волатильность** представляет собой меру риска использования финансового инструмента за заданный промежуток времени. Иными словами, волатильность показывает меру изменчивости и часто измеряется в процентах или долях. Предсказание волатильности, например, позволяет инвесторам определить риск приобретения финансового инструмента.

Для таких данных с непостоянной дисперсией был разработан ещё один класс моделей — семейство моделей авторегрессионной условной гетероскедастичности, или `ARCH` (`Autoregressive Conditional Heteroscedastic Model`).

## ARCH И GARCH

Давайте сначала разберёмся, как устроена модель `ARCH`, а затем поговорим о том, как правильно её применять.

Ниже представлен график ошибок предсказания некоторого временного ряда. Что с ним не так?

<img src=m13_img12.png>

В прошлом модуле мы обсуждали, что в хорошей модели, учитывающей все зависимости, в остатках должен получиться белый шум. Однако белый шум должен иметь одинаковое распределение на всех участках временного ряда, а значит, иметь одинаковые математическое ожидание и дисперсию.

Если на графике выше с математическим ожиданием всё в порядке (среднее значение в нуле), то к дисперсии есть вопросы, так как видно, что она различна на разных промежутках (разброс в разные годы отличен).

$\sigma^2(t)=a+\sum_{i=1}^{q}b_ir_{t-1}^2$



Здесь $q$ — количество слагаемых, которые влияют на текущее значение, а $b$ — весовые коэффициенты, которые влияют на степень значимости предыдущих изменений дисперсии ($r$). То есть волатильность моделируется в виде суммы константы ($a$ — базовая волатильность, константа) и линейной функции абсолютных значений изменения нескольких последних цен.

>**Обратите внимание**: из-за квадрата в формуле волатильность предсказывается только в абсолютном значении (то есть по модулю).

Чуть позднее другой учёный, Тим Боллерслерв, предложил обобщённую концепцию модели `ARCH` — `GARCH` (`Generalized Autoregressive Conditional Heteroscedastic Model`). Модель предполагала, что на изменчивость дисперсии влияют не только предыдущие изменения показателей, но и предыдущие оценки дисперсии (значение дисперсии).

$\sigma^2(t)=a+\sum_{i=1}^{q}b_ir_{t-1}^2+\sum_{i=1}^{p}c_i \sigma_{t-1}^2$

* первая часть формулы — ARCH-модель;

* $p$ — количество оценок, предшествующих текущей, которые влияют на текущее значение;

* $c$ — это весовые коэффициенты, которые влияют на степень значимости предыдущих дисперсий ($\sigma^2$).
Как и в случае AR-моделей, эти коэффициенты настраиваются автоматически в процессе обучения.

[ARCH и GARCH](https://arch.readthedocs.io/en/latest/univariate/introduction.html) — наиболее популярные модели, используемые при прогнозе волатильности. Существуют и их модернизации, такие как A-GARCH, E-GARCH и многие другие. В будущем, если вам придётся тесно работать с финансовыми временными рядами, вы сможете самостоятельно познакомиться с этими моделями.

## КОГДА ПРИМЕНЯТЬ ARCH И GARCH?

* Когда ряд похож на белый шум, но при этом в нём **присутствует гетероскедастичность**.
Чтобы определить, является ли ряд гетероскедастичным (с меняющейся дисперсией), можно отобразить его квадраты на графике и понаблюдать за поведением дисперсии.

* Когда после применения AR-модели **остатки (ошибки модели) тоже являются гетероскедастичными**. В этом случае вы также можете прогнозировать дисперсию ошибок и использовать её в итоговом предсказании. Для этого необходимо суммировать результаты AR-модели с результатами ARCH.


<img src=m13_img13.png>

## ДОПОЛНИТЕЛЬНО:

* [Задача](https://medium.com/@ranjithkumar.rocking/time-series-model-s-arch-and-garch-2781a982b448) моделирования остатков.

* [Короткая лекция](https://medium.com/@ranjithkumar.rocking/time-series-model-s-arch-and-garch-2781a982b448) с разбором математики ARCH и GARCH.

## Валидация временных рядов

Последовательное разбиение может выглядеть так (`train`, `test`):

<img src=m13_img14.png>

То есть в случае с данными на графике выше правильно будет разбить их как `train = [0,1,2,3,4,5]` и `test = [6,7,8,9]`. Соотношение объёма выборок, конечно, может быть другим, но главное — выборки должны идти последовательно.

## АНАЛОГ КРОСС-ВАЛИДАЦИИ ДЛЯ ВРЕМЕННЫХ РЯДОВ

Если использовать обычную кросс-валидацию, информация из будущего снова просочится в алгоритм. Поэтому вместо кросс-валидации используются:

* **walk forward validation**;

* **множественное разбиение**.

<img src=m13_img15.png width=800>

>Обратите внимание на изображения выше. В обоих случаях мы также соблюдаем последовательность в разбиении — сравните с обычной кросс-валидацией (слева). Оба разбиения действуют по схожему принципу — делят выборки упорядоченно, чтобы подвыборка test всегда выбиралась после подвыборок train. Отличием будет то, что в walk forward validation размер обучающих выборок во всех фолдах (разбиениях) будет одинаковым, а во множественном разбиении в каждом новом фолде данные обучающей выборки накапливаются.

1. Разбейте временной ряд на три набора (`3 train + 3 test`). Для этого воспользуйтесь классом [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) из `sklearn.model_selection` с параметрами `n_splits=3` и `test_size=7` (или `max_train_size=df.sales.count()-7`). Наборы данных можно получить, вызвав метод `split` (временной ряд) у инициализированного `TimeSeriesSplit`.

In [6]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

df = pd.read_csv("sp500.zip", parse_dates=["Date"], index_col=["Date"])

tscv = TimeSeriesSplit(n_splits=3, test_size=7)

train_test_groups = tscv.split(df)
train_test_groups

<generator object TimeSeriesSplit.split at 0x000001CAD0ECBCF0>

In [22]:
train_test_groups = tscv.split(df)
i = 0
train = []
test = []
for train_index, test_index in train_test_groups:
    print("TRAIN size:", len(train_index), "TEST size:", len(test_index))
    train.append(df.iloc[train_index])
    test.append(df.iloc[test_index])
    
train[-1], test[-1]

TRAIN size: 6671 TEST size: 7
TRAIN size: 6678 TEST size: 7
TRAIN size: 6685 TEST size: 7


(                    spx
 Date                   
 1994-01-06   467.119995
 1994-01-07   469.899994
 1994-01-10   475.269989
 1994-01-11   474.130005
 1994-01-12   474.170013
 ...                 ...
 2019-08-15  2847.600098
 2019-08-16  2888.679932
 2019-08-19  2923.649902
 2019-08-20  2900.510010
 2019-08-21  2924.429932
 
 [6685 rows x 1 columns],
                     spx
 Date                   
 2019-08-22  2922.949951
 2019-08-23  2847.110107
 2019-08-26  2878.379883
 2019-08-27  2869.159912
 2019-08-28  2887.939941
 2019-08-29  2924.580078
 2019-08-30  2926.459961)

## Файл `project-Part3.ipynb`

В скринкасте мы получаем предсказания с помощью `model_results.forecast()`.

Любое обращение к методу `forecast()` возвращает объект `ARCHModelForecast` с тремя основными атрибутами:

* `.mean` — условное среднее значение ряда;

* `.variance` — условная дисперсия прогноза;

* `.residual_variance` — прогнозируемая условная дисперсия остатков. `Residual_variance` будет отличаться от `variance` в случаях, когда значения ряда (не дисперсии, а именно сами значения) имеют зависимость от себя в прошлом. В скринкасте для предсказания дисперсии мы используем `residual_variance`, так как в нашем случае ряд стационарен и `residual_variance` и variance возвращают одинаковые значения.

***

## Другие методы предсказания временных рядов

## PROPHET

>`Prophet` — это метод прогнозирования данных временных рядов на основе AR-модели, в которой учтены годовая, еженедельная и ежедневная сезонности, а также эффекты праздничных дней.

Prophet лучше всего работает с временными рядами, которые имеют сильные сезонные эффекты, а данные накоплены за несколько сезонов. Алгоритм устойчив к отсутствующим данным и сдвигам в тренде и обычно хорошо справляется с выбросами.

[Prophet](https://facebook.github.io/prophet/docs/quick_start.html#python-api) - библиотека с открытым исходным кодом, выпущенная командой Facebook Core Data Science. Для загрузки метод также доступен в [PyPI](https://pypi.org/project/prophet/) (через pip install).

Prophet следует `API` модели `sklearn`. Это значит, что мы можем пользоваться им и его методами так же, как и в случае с моделями из `sklearn`: для инициализации модели создаётся экземпляр класса Prophet (`myModel = Prophet()`), а затем вызываются его методы обучения (`.fit()`) и прогнозирования (`.predict()`). Входные данные для методов Prophet должны представлять собой датафрейм с двумя столбцами — `DS` и `Y`.

Столбец `DS` (отметка даты) должен иметь временной формат (`DateTime`), например ГГГГ-ММ-ДД — для даты или ГГГГ-ММ-ДД ЧЧ:ММ:СС — для отметки времени.
Столбец `Y` должен быть числовым и представлять измерение, которое мы хотим спрогнозировать.

## NEURALPROPHET

>[NeuralProphet](https://neuralprophet.com/) — основанная на [PyTorch](https://pytorch.org/) усовершенствованная и более сложная модель, которая комбинирует в себе преимущества традиционных моделей для анализа временных рядов и методов глубокого обучения. `NeuralProphet` можно установить с помощью pip (`pip install neuralprophet`).

Вы ещё не знакомы с нейронными сетями, поэтому мы не будем углубляться в подробности реализации данного алгоритма. Тем не менее вы можете смело использовать эту библиотеку, так как запустить её в несколько шагов «из коробки» не представляет большого труда.

>Познакомиться и поэкспериментировать с NeuralProphet вы можете, разбирая примеры из официального репозитория проекта на GitHub. Например, обратите внимание на решение [задачи](https://github.com/ourownstory/neural_prophet/blob/main/tutorials/application-example/energy_hospital_load.ipynb) прогнозирования энергетической нагрузки на больницу в Сан-Франциско.

В [этой русскоязычной статье](https://habr.com/ru/company/otus/blog/555700/) представлен более подробный разбор алгоритма на примере решения задачи предсказания ежедневных просмотров страниц на Wikipedia, но уже с помощью `NeuralProphet`.

## ДОПОЛНИТЕЛЬНО:

Если вас заинтересовали библиотеки `Prophet` и `NeuralProphet`, рекомендуем обратить внимание [на эту статью](https://towardsdatascience.com/prophet-vs-neuralprophet-fc717ab7a9d8). Помимо математического описания, там сравниваются результаты работы этих алгоритмов на реальной задаче.